# The Issue Description

Over fifty thousands people in Belarus have been arrested during last two years due to their political position. Even now dozens are appeared in prison every day. Police never tells names of apprehended as well as any details related to current status and location of prisoners. In this case your family and friends wouldn't know even where you are.

Since 08.2020 huge number of volunteers have been collecting the information and regularly post it in Telegram channels. Almost every day lists with data of recently arrested people are published. 

Sometimes you take a look at such lists and attempt to find someone whom is known for you: relatives, friends, colleagues, schoolmates, etc. It could be extremely useful in case if you are able to sent some food and clothes or help somehow the prisoner's family.

I've created fast solution of very simple Telegram bot for matching data row per row by names and surnames:
- You need to list surname, name, patronymic and birth year of all persons you are interested in to separate .csv;
- Upload it to local database;
- Forward a message from Telegram channels of volunteers to the bot;
- Notice the result: the pairs of matched rows, one from your list and another from list of prisoners. 

# Data Preparation

In [4]:
import sqlite3 as sl
import pandas as pd
import typing as tp

In [5]:
def insert_to_db(
    sql_drop: str
    , sql_create: str
    , sql_insert: str
    , df_ins: tp.Any
    , con: tp.Any
    , cur: tp.Any
) -> str:
    """
    UDF for creating tables and inserting data using sqlite3
    """
    
    #I've noticed SQLITE usually convert int data to blob type, so let's convert it
    for i, column in enumerate(df_ins.columns):
        if df_ins[column].dtype == 'int64':
            df_ins[column] = df_ins[column].astype(object)
    
    #connection to db
    with con:
        try:
            cur.execute(sql_drop)
            cur.execute(sql_create)

            #inserting data to table
            for i in range(df_ins.shape[0]):
                data = []
                data.append(tuple(df_ins.loc[i]))            
                cur.executemany(sql_insert, data)  

        except sl.DatabaseError as err:       
            return("Error: ", err)
        else:
            con.commit()
    return("Data was successfully inserted")

In [7]:
sql_drop = "drop table if exists main_line;"
sql_create = """
            create table main_line
            (
            id integer not null primary key autoincrement
            , year inet
            , surname text
            , name text
            , patronymic text             
            );
            """
sql_insert = "insert into main_line(surname, name, patronymic, year) values (?, ?, ?, ?);"

con = sl.connect('my_base.db')
cur = con.cursor()

df = pd.read_excel("db_dataset.xlsx")

insert_to_db(sql_drop, sql_create, sql_insert, df_ins = df, con = con, cur = cur)

'Data was successfully inserted'

# Bot

The body which is run from prompt.

In [ ]:
import sqlite3 as sl
import pandas as pd
import os
from fuzzywuzzy import fuzz

from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher
from aiogram.utils import executor
from random import randrange

from config import TOKEN

con = sl.connect('my_base.db')

#target frame for finding
df_out = pd.read_sql('''
select * 
from main_line
''', con)

#divide input message by separate strings
def partial_string(s):
    indexes = []
    indexes.append(0)
    sub = []
    string = []
    i = 0
    
    if "\n" in s:
        
        while i < len(s):

            if s[i] != "\n":
                i = i + 1
            else:
                indexes.append(i)
                string.append(s[indexes[-2]:indexes[-1]])
                i = i + 1
        
        string.append(s[indexes[-1]:-1])

    else:
        string = s
    
    return string

#find rows from target frame in input message
def find(string, df_find):
    final_result = []
    new_string = []
    
    for i in range(len(string)):
        new_string.append(string[i].replace(" ", "").lower()) 
    
    for j in range(df_find.shape[0]):

        person_bef = str(df_find.at[j, "surname"]) + " " + str(df_find.at[j, "name"]) + " " + str(df_find.at[j, "patronymic"]) + " " + str(df_find.at[j, "year"])
        
        person = str(df_find.at[j, "surname"].lower()) + str(df_find.at[j, "name"].lower())
        
        for k in range(len(string)):
            result = fuzz.partial_ratio(person, new_string[k])
                
            if result > 95:
                final_result.append(person_bef + ": " + (string[k])) 
            else:
                continue

    if len(final_result) == 0:
        ret = ""
    else:
        
        ret = final_result
    return ret

bot = Bot(token = TOKEN)
dp = Dispatcher(bot)

@dp.message_handler()
async def echo_message(msg: types.Message):
    await bot.send_message(msg.from_user.id, "Processing...")
    await bot.send_message(msg.from_user.id, "Matches found: " + str(len(find(partial_string(msg.text), df_out))))
    await bot.send_message(msg.from_user.id, find(partial_string(msg.text), df_out))
    
if __name__ == '__main__':
    try:
        executor.start_polling(dp)
    except:
        os._exit(0)